<a href="https://colab.research.google.com/github/PabloGorSan/AlgortimosDeOptimizacion/blob/main/Copia_de_Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Pablo Gordillo Sánchez   <br>
Url: https://github.com/PabloGorSan/AlgortimosDeOptimizacion/blob/main/Copia_de_Seminario_Algoritmos.ipynb<br>
Problema:

Combinar cifras y operaciones

Descripción del problema:(copiar enunciado)

- El problema consiste en analizar el siguiente problema y diseñar un algoritmo que lo resuelva.
- Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las
operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/)
- Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una cantidad
dada. Un ejemplo sería para obtener el 4:
4+2-6/3*1 = 4
- Debe analizarse el problema para encontrar todos los valores enteros posibles planteando las siguientes cuestiones:
 - ¿Qué valor máximo y mínimo se pueden obtener según las condiciones del problema?
 - ¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo?
                                        

In [159]:
import bisect
import random

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>







Para este problema debemos definir previamente qué significa que no tenga restricciones, ya que podemos entenderlo de distinta manera y obtener una cantidad de posibilidades diversas. Las restricciones que plantea el problema son:
- Debemos combinar números y operaciones alternativamente
- Sin repetir ninguno de ellos.

Empezando por la segunda restricción, si esta no se cumpliera, el espacio de búsqueda para este problema sería infinito, ya que podríamos siempre seguir creando nuevas posibilidades añadiendo una nueva cifra o una nueva operación según corresponda. Por tanto, no vamos a tomarlo tanto como una restricción sino como una característica de nuestro problema (igual que utilizar las cifras y las operaciones básicas).

La primera restricción es más interesante. Anular esta restricción nos sigue limitando el espacio de búsqueda, aunque añade combinaciones no validas (como 2*/3). Teniendo en cuenta las no validas, las posibilidades totales serían igual a 13!, ya que sería una permutación sin repetición de 13 signos (9 dígitos y 4 operaciones).


¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?

Al igual que hemos deducido el número de posibilidades en el apartado anterior, podemos aplicar el mismo razonamiento para obtener el total de posibilidades teniendo en cuenta las restricciones.
En un comienzo debemos elegir una de entre 9 cifras para probar, luego una entre 4 operaciones, una entre 8 cifras, una entre 3 operaciones... hasta quedarnos sin operaciones, por lo que ya solo podremos añadir una cifra más. Por tanto el número de posibilidades es 9 * 4 * 8 * 3 * 7 * 6 * 2 * 5 * 1 = 9! = 362880.

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Cada solución está formada por una cadena de 9 caracteres, que se evalúa a un valor. Estas cadenas se van a explorar en forma de árbol, ya que plantearemos la construcción de estas soluciones como una "elección" del carácter a añadir en cada momento. Tal y como se pide en el enunciado, la solución final deberá estar constituida por una lista de valores enteros evaluados de estas cadenas. A poder ser, se obtendrá la lista de valores ordenada, ya que permitirá obtener el mínimo y el máximo de manera sencilla, además de realizar la comprobación de que se han obtenido todos los valores intermedios.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

En este problema no se busca optimizar un valor, como sí podría ser en un problema de asignación o de búsqueda del camino más corto, sino realizar una búsqueda de los caminos válidos en un árbol. Por tanto, esta función objetivo se podría definir como "Diferencia entre valores enteros encontrados y los esperados", siendo cada elemento faltante o sobrante sumado. Si se planteara así, para lo cual se deberían conocer previamente los valores esperados, el problema se trataría de un problema de minimización, donde se buscaría reducir el número de discrepancias entre ambos conjuntos. Esto puede ser interesante en algoritmos donde se sacrifique la exploración completa del árbol a cambio de eficiencia, ya que podrían saltarse soluciones válidas.

Diseña un algoritmo para resolver el problema por fuerza bruta

El algoritmo de fuerza bruta es bastante simple al implementarse aplicando recursividad. Este algoritmo tomará una cifra de la lista de cifras de entrada. Si hay operaciones de entrada, el algoritmo tomará una operación, los añadirá a la cadena a evaluar y se llamará recursivamente con quitando ambos elementos de sus respectivas listas. Este proceso se repite hasta que no queden operaciones, en cuyo caso se evaluarán todas las cadenas posibles añadiendo la última cifra. Tras cada evaluación, la cual conlleva que se haya explorado un nodo hoja, se comprobará si la solución es entera. De ser así se añadirá ordenadamente a la lista de soluciones. De no ser entera se descartará.

Tras esto, se iterará sobre las soluciones para comprobar que todos los valores intermedios han sido encontrados.

In [131]:
cifras = [str(x) for x in range(1, 10)]
operaciones = ['+', '-', '*', '/']

def fuerzaBrutaRestricciones(cifras, operaciones, cadena):
  valores = {
      'posibilidades' : 0,          #Nodos hoja explorados
      'max' : -9999999,             #Valor máximo calculado
      'min' : 9999999,              #Valor mínimo calculado
      'valores enteros' : [],       #Valores enteros calculados
      'valores intermedios' : True  #Todos los valores intermedios han sido encontrados
  }
  calcularValoresEnteros(cifras, operaciones, "", valores) 
  return encontrarValoresIntermedios(valores) 

def calcularValoresEnteros(cifras, operaciones, cadena, valores): #Obtiene todos los valores enteros
  for c in cifras:
    if len(operaciones) == 0: #Si no quedan operaciones que añadir a la cadena se prueban todas las cifras restantes
      valor = eval(cadena + c)
      valores['posibilidades'] += 1
      if valor.is_integer():
        if valor not in valores['valores enteros']:
          bisect.insort(valores['valores enteros'], valor) #Inserción ordenada en lista ordenada O(N)
        if valor > valores['max'] : valores['max'] = valor
        if valor < valores['min'] : valores['min'] = valor
    else:  
      for o in operaciones: #Si quedan más operaciones se añaden
        cifrasMenos1 = cifras.copy()
        cifrasMenos1.remove(c)
        operacionesMenos1 = operaciones.copy()
        operacionesMenos1.remove(o)
        calcularValoresEnteros(cifrasMenos1, operacionesMenos1, cadena + c + o, valores) #LLamada recursiva eliminando la cifra y la operación seleccionadas

def encontrarValoresIntermedios(valores): #Comprueba que todos los valores intermedios han sido encontrados
    i = 0
    while i < len(valores['valores enteros'])-1 and valores['valores intermedios']:
      if abs(valores['valores enteros'][i] - valores['valores enteros'][i]+1) != 1: valores['valores intermedios'] = False #Comprueba que todos los valores están a una unidad de distancia
      else: i += 1

    return valores

valores = fuerzaBrutaRestricciones(cifras, operaciones, "")

In [81]:
for v in valores.keys():
  print(v + " " + str(valores[v]));

posibilidades 362880
max 77.0
min -69.0
valores enteros [-69.0, -68.0, -67.0, -66.0, -65.0, -64.0, -63.0, -62.0, -61.0, -60.0, -59.0, -58.0, -57.0, -56.0, -55.0, -54.0, -53.0, -52.0, -51.0, -50.0, -49.0, -48.0, -47.0, -46.0, -45.0, -44.0, -43.0, -42.0, -41.0, -40.0, -39.0, -38.0, -37.0, -36.0, -35.0, -34.0, -33.0, -32.0, -31.0, -30.0, -29.0, -28.0, -27.0, -26.0, -25.0, -24.0, -23.0, -22.0, -21.0, -20.0, -19.0, -18.0, -17.0, -16.0, -15.0, -14.0, -13.0, -12.0, -11.0, -10.0, -9.0, -8.0, -7.0, -6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0]
valores int

Calcula la complejidad del algoritmo por fuerza bruta

En este algoritmo vamos a considerar como operación unitaria la evaluación de una cadena, es decir, los nodos hoja que se han explorado en el árbol que forman las variaciones de estos elementos. Ya que se trata de un algoritmo iterativo y recursivo, podemos calcular la complejidad de la siguiente forma:

Siendo C el tamaño de la lista de cifras y siendo O el tamaño de la lista de operaciones, con C > O el número de operaciones realizadas sería:

- C , O = 0
- C * O * T((C-1), (O-1)) * ... * T((C-O), 0) , O > 0

Por tanto, podemos simplificar esta operación como:

- C * O * (C-1) * (O-1) * ... * C = (C! * O!) / (C-O-1)!

En el peor caso, donde C = O + 1 ya que siempre debe haber un digito almenos más que operaciones, la complejidad sería O(C!*(C-1)!). Es decir, tiene complejidad factorial, por lo que no es un algoritmo viable para su uso con valores altos.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta por qué crees que mejora el algoritmo por fuerza bruta

Podemos imaginar el espacio de búsqueda de este problema como un grafo que se va ampliando en forma de árbol, pero con posibilidad de que aparezcan nodos compartidos entre sus ramas. Esto ocurre por la existencia de distintas operaciones matemáticas que tienen el mismo resultado (2+3 = 6-1). Por ello, podemos plantear este problema como un problema resoluble mediante Ramificación y Poda del grafo, ya que podemos intentar recordar los nodos comunes ya visitados para no tener que volver a explorarlos. Para ello, cada vez que añadamos una cifra a la cadena, añadiremos una clave a un diccionario con la evaluacion de la cadena actual y las cifras que van a explorarse; y como valor las operaciones que a explorar. Así, si se vuelve a un valor que ya se había calculado, junto con unas cifras y operaciones ya calculadas, podremos omitir esa búsqueda.

In [177]:
cifras = [str(x) for x in range(1, 10)]
operaciones = ['+', '-', '*', '/']

def programacionDinamicaRestricciones(cifras, operaciones, cadena):
  valores = {
    'posibilidades' : 0,
    'max' : -9999999,
    'min' : 9999999,
    'valores enteros' : [],
    'valores intermedios' : True
  }
  visitados = {} #Inicializamos el diccionario
  calcularValoresEnterosPD(cifras, operaciones, "", valores, visitados)
  return encontrarValoresIntermedios(valores), visitados

def calcularValoresEnterosPD(cifras, operaciones, cadena, valores, visitados):
  for c in cifras:
    valor = eval(cadena + c);
    if len(operaciones) == 0: #Si no quedan operaciones que añadir a la cadena se prueban todas las cifras restantes
      valores['posibilidades'] += 1
      if float(valor).is_integer():
        if valor not in valores['valores enteros']:
          bisect.insort(valores['valores enteros'], valor)
        if valor > valores['max'] : valores['max'] = valor
        if valor < valores['min'] : valores['min'] = valor
    else: #Si quedan operaciones se comprueba en el diccionario si la tripleta valor, cifras, operaciones ya ha sido explorada
      if (str(valor) + ", " + str(cifras)) in visitados.keys() and str(operaciones) in visitados[str(valor) + ", " + str(cifras)]:
        return #Si se encuentra termina esta exploración
      else: #Si no se encuentra se añade el par valor, cifras al diccionario y se continua normalmente
        visitados[str(valor) + ", " + str(cifras)] = []
        for o in operaciones:
          cifrasMenos1 = cifras.copy()
          cifrasMenos1.remove(c)
          operacionesMenos1 = operaciones.copy()
          operacionesMenos1.remove(o)
          calcularValoresEnterosPD(cifrasMenos1, operacionesMenos1, cadena + c + o, valores, visitados)
        visitados[str(valor) + ", " + str(cifras)].append(str(operaciones)) #Cuando se evaluen todas las permutaciones se añade al diccionario
    

valores, visitados = programacionDinamicaRestricciones(cifras, operaciones, "")


In [158]:
for v in valores.keys():
  print(v + " " + str(valores[v]));

posibilidades 122365
max 77.0
min -69.0
valores enteros [-69.0, -68.0, -67.0, -66.0, -65.0, -64.0, -63.0, -62.0, -61.0, -60.0, -59.0, -58.0, -57.0, -56.0, -55.0, -54.0, -53.0, -52.0, -51.0, -50.0, -49.0, -48.0, -47.0, -46.0, -45.0, -44.0, -43.0, -42.0, -41.0, -40.0, -39.0, -38.0, -37.0, -36.0, -35.0, -34.0, -33.0, -32.0, -31.0, -30.0, -29.0, -28.0, -27.0, -26.0, -25.0, -24.0, -23.0, -22.0, -21.0, -20.0, -19.0, -18.0, -17.0, -16.0, -15.0, -14.0, -13.0, -12.0, -11.0, -10.0, -9.0, -8.0, -7.0, -6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0]
valores int

In [156]:
len(visitados)

25284

(*)Calcula la complejidad del algoritmo 

La complejidad de este algoritmo a nivel computacional no ha descendido. De base, se siguen debiendo explorar todos los nodos si no se repitiesen nunca valores calculados en las cadenas. Sin embargo, la poda nos permite no tener que explorar gran parte del árbol. Viendo los datos obtenidos tras la ejecución, los nodos hoja explorado han sido 122365, un 33.72% de los nodos explorados mediante fuerza bruta.

Sin embargo hay que destacar que, al añadirse el diccionario es necesario realizar búsquedas en la misma durante la ejecución, además de que este ocupa un espacio en memoria que pueda llegar a ser considerable.

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Tanto el número de cifras de entrada como las operaciones pueden añadirse de manera aleatoria. Hay que tener en cuenta, eso sí, que si se quieren añadir nuevas operaciones éstas pueden que no sean correctas en todos los casos (como una raíz cuadrada de un número negativo).

In [211]:
cifras = [str(x) for x in range(1, 20)] #Añadimos cifras del 1 al 19
operaciones = ['+', '-', '*', '/','%'] #Añadimos la potencia y el resto de una división

cifrasRandom = random.sample(cifras, 10) #Añadimos un número distinto de cifras, pueden ser más de 9 si contamos el 10 como una sola cifra
operacionesRandom = random.sample(operaciones, 4) #Utilizamos cualquier combinación de 4 operaciones. Se recomienda probar con 5 (aprox 15 segundos de ejecución).
print(cifrasRandom)
print(operacionesRandom)

['5', '18', '4', '6', '19', '13', '10', '8', '14', '3']
['%', '-', '/', '+']


Aplica el algoritmo al juego de datos generado

In [212]:
valores, visitados = programacionDinamicaRestricciones(cifrasRandom, operacionesRandom, "")

In [213]:
for v in valores.keys():
  print(v + " " + str(valores[v]));

posibilidades 196506
max 37.0
min -17.0
valores enteros [-17.0, -16.0, -15.0, -14.0, -13.0, -12.0, -11.0, -10.0, -9.0, -8.0, -7.0, -6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 35.0, 36.0, 37.0]
valores intermedios True


In [210]:
len(visitados)

163844

Haciendo pruebas con el código y los datos aleatorios podemos ver cosas muy interesantes y que ya se han ido preveyendo en anteriores apartados. Aumentar el número de operaciones es más costoso que aumentar el número de cifras, ya que al añadir una nueva operación se permiten elegir dos elementos más en la cadena. Sin embargo permite seguir realizando la poda de manera eficiente, mitigando el número de nodos a explorar.

También podemos comprobar que añadir valores en un mayor rango (como de 1 a 20) hace que haya más repeticiones que almacenar en nuestro diccionario, por lo que la eficiencia de nuestro algoritmo varía en función de los propios valores de entrada.

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Principalmente he utilizado los apuntes de nuestra asignatura, más allá de consultas puntuales sobre el lenguaje Python. 

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Este problema tiene un espacio de búsqueda muy alto, por lo que es necesario encontrar algoritmos que realicen esta búsqueda de valores de manera eficiente. El algoritmo de ramificación y poda permite mitigar el crecimiento del espacio de búsqueda como hemos podido ver, pero sigue teniendo una complejidad muy alta. Podríamos seguir aplicando este tipo de podas (por ejemplo intentando predecir si los valores que se obtendrán por una rama pueden ser enteros o serán decimales). Sin embargo esto no creo que sea suficiente. 

Ya que hay muchas cadenas que obtienen el mismo valor con variaciones diferentes de caracteres, puede que una búsqueda heurística sea interesante de plantear. Así, podríamos seguir descartando ramas del árbol si preveemos que no obtendremos valores muy distintos por ese camino. Por otro lado eso ya supondría la posibilidad de no obtener todos los valores enteros posibles, además de que el heurístico es difícil de definir en rangos de valores muy grandes.

También se podría plantear el uso de algoritmos genéticos para resolver el problema. Para ello podríamos definir soluciones candidatas donde N genes pertenezcan a las N cifras seleccionadas y M genes a las M operaciones. Para cada solución podemos obtener el número de valores enteros que podemos calcular y quedarnos con los individuos que en conjunto obtengan un mayor número de valores enteros descubiertos o de discrepancias con el conjunto esperado si contamos con él. 